[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [4]:
# Read dataset with extra params sep='\t', encoding="latin-1"
#data = pd.read_csv('./labeledTrainData.csv')
data = pd.read_csv('./labeledTrainData.csv', sep='\t', encoding="latin-1")
data = pd.DataFrame(data = data)

In [41]:
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## 2. Preprocessing

In [22]:
# stop words
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\TONY
[nltk_data]     DANG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)  # Your code here
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text
print(preprocessor('Create some random texts for testing <>><<?//:-) ?'))

create some random texts for testing  :)


In [19]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer_porter(text):
    token = [porter.stem(word) for word in text.split()]
    return token
#    return [] # Your code here

In [27]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X = data['review']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 66)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [30]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop, tokenizer=tokenizer_porter, preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0, max_iter = 120))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [36]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Using Test dataset to evaluate model
prediction = clf.predict(X_test)
# classification_report
print(accuracy_score(y_test, prediction))

print(classification_report(y_test, prediction))
# confusion matrix
print(confusion_matrix(y_test, prediction))

0.882
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      3793
           1       0.86      0.91      0.88      3707

   micro avg       0.88      0.88      0.88      7500
   macro avg       0.88      0.88      0.88      7500
weighted avg       0.88      0.88      0.88      7500

[[3255  538]
 [ 347 3360]]


## 5. Export Model 

In [40]:
twits = [
    "This is really bad",
    "I love this!",
    ":)",
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

This is really bad --> Negative, Positive = [0.99475572 0.00524428]
I love this! --> Negative, Positive = [0.01698334 0.98301666]
:) --> Negative, Positive = [0.18086828 0.81913172]


In [37]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open(os.path.join('logisticRegression.pkl'), 'wb'), protocol=4)